In [1]:
import pandas as pd
import altair as alt
from sublimpy import utils
alt.data_transformers.enable('json')
import datetime as dt
import numpy as np

ls ~/Development/data/sublimationofsnow/ | grep csv

In [2]:
src = pd.read_csv(
    "~/Development/data/sublimationofsnow/monthly_planar_fits.csv", delim_whitespace=True
).groupby(['height', 'tower']).mean().sort_values('tilt').reset_index()

src[src.height.isin(['3','10'])]

/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_37803/941560937.py:1: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  src = pd.read_csv(


,height,tower,month,a,b,c,tilt,tiltaz,W_f_1,W_f_2,W_f_3
1,3,ue,5.5,0.001482,-0.000199,0.000215,0.028113,-47.998771,0.000199,-0.000215,1.000000
2,3,c,5.5,0.000805,0.000034,-0.000078,0.031190,-25.177788,-0.000034,0.000078,1.000000
3,3,d,5.5,0.008837,-0.000067,0.000064,0.034341,-32.139706,0.000067,-0.000064,1.000000
7,10,uw,5.5,-0.000085,-0.000048,0.000637,0.055466,-38.037183,0.000048,-0.000637,0.999999
8,3,uw,5.5,0.000648,-0.000253,0.000525,0.062453,-54.218484,0.000253,-0.000525,0.999999
9,10,d,5.5,0.041599,0.000271,0.000619,0.066754,-69.713736,-0.000271,-0.000619,0.999999
14,10,ue,5.5,0.035018,0.000175,0.000095,0.105877,-10.622671,-0.000175,-0.000095,0.999998
15,10,c,5.5,-0.055529,0.000526,0.001037,0.114147,-101.547371,-0.000526,-0.001037,0.999997


In [3]:
df_nopf = pd.read_parquet("tidy_df_20221101_20230619_noplanar_fit.parquet")
df_pf = pd.read_parquet("tidy_df_20221101_20230619_planar_fit.parquet")
df_pfmp = pd.read_parquet("tidy_df_20221101_20230619_planar_fit_multiplane.parquet")

df_nopf = utils.modify_df_timezone(df_nopf, 'UTC', 'US/Mountain')
df_pf = utils.modify_df_timezone(df_pf, 'UTC', 'US/Mountain')
df_pfmp = utils.modify_df_timezone(df_pfmp, 'UTC', 'US/Mountain')

df = pd.concat([
    df_nopf.query("measurement == 'w'").assign(type = "nopf"),
    df_pf.query("measurement == 'w'").assign(type = "pf"),
    df_pfmp.query("measurement == 'w'").assign(type = "pfmp"),
])
df = df[(df.time > '20221130') & (df.time < '20230509')]

In [4]:
src_pfmp = df.query("type == 'pfmp'").set_index('time')
src_pfmp = src_pfmp.groupby([src_pfmp.index.floor('30T').time, 'height', 'tower', 'type'])[['value']].mean()
src_pfmp.index = src_pfmp.index.set_names(['time','height','tower', 'type'])
src_pfmp = src_pfmp.reset_index()
src_pfmp['time'] = src_pfmp['time'].apply(lambda t: dt.datetime.combine(dt.date(2023,1,1),t))

src_nopf = df.query("type == 'nopf'").set_index('time')
src_nopf = src_nopf.groupby([src_nopf.index.floor('30T').time, 'height', 'tower', 'type'])[['value']].mean()
src_nopf.index = src_nopf.index.set_names(['time','height','tower', 'type'])
src_nopf = src_nopf.reset_index()
src_nopf['time'] = src_nopf['time'].apply(lambda t: dt.datetime.combine(dt.date(2023,1,1),t))

src_pf = df.query("type == 'pf'").set_index('time')
src_pf = src_pf.groupby([src_pf.index.floor('30T').time, 'height', 'tower', 'type'])[['value']].mean()
src_pf.index = src_pf.index.set_names(['time','height','tower', 'type'])
src_pf = src_pf.reset_index()
src_pf
src_pf['time'] = src_pf['time'].apply(lambda t: dt.datetime.combine(dt.date(2023,1,1),t))

/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_37803/1951778730.py:2: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  src_pfmp = src_pfmp.groupby([src_pfmp.index.floor('30T').time, 'height', 'tower', 'type'])[['value']].mean()
/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_37803/1951778730.py:8: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  src_nopf = src_nopf.groupby([src_nopf.index.floor('30T').time, 'height', 'tower', 'type'])[['value']].mean()
/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_37803/1951778730.py:14: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  src_pf = src_pf.groupby([src_pf.index.floor('30T').time, 'height', 'tower', 'type'])[['value']].mean()


In [8]:
line = alt.Chart().transform_calculate(y = '0').mark_rule().encode(y='y:Q')
plot = alt.Chart().mark_line().encode(
    alt.X('hoursminutes(time):T'),
    alt.Y('value:Q').title('Vertical wind velocity (m/s)'),
    alt.Color('height:O').scale(scheme='turbo').title('Height (m)'),
)
compressed_plot = alt.Chart().mark_line().encode(
    alt.X('hoursminutes(time):T'),
    alt.Y('value:Q').title('Vertical wind velocity (m/s)'),
    alt.Color('height:O').scale(scheme='turbo').title('Height (m)'),
    alt.StrokeDash('tower:N')
)

plot = (
    alt.layer(
        line, compressed_plot, data = src_nopf.query("height > 1")
    ).properties(width = 300, height = 150, title='Geographic coordinates')
    &
    alt.layer(
        line, compressed_plot, data = src_pfmp.query("height > 1")
    ).properties(width = 300, height = 150, title='Streamline coordinates')
)

In [14]:
plot.save("/Users/elischwat/Downloads/slec_vs_streamline_plot.png", dpi=200)
plot.display(renderer='svg')

alt.VConcatChart(...)

In [6]:
alt.layer(
    line, plot, data = src_pfmp
).properties(width = 150, height = 150).facet('tower:N', title='Seemingly local earth coordinates').display(renderer='svg')


alt.FacetChart(...)

In [13]:
(
    alt.layer(
        line, plot, data = src_pfmp
    ).properties(width = 150, height = 150).facet('tower:N', title='Streamwise coordinates')
    &
    alt.layer(
        line, plot, data = src_pf
    ).properties(width = 150, height = 150).facet('tower:N', title='Seemingly local earth coordinates')

    &
    alt.layer(
        line, plot, data = src_nopf
    ).properties(width = 150, height = 150).facet('tower:N', title='Geographic coordinates')

).display(renderer='svg')

alt.VConcatChart(...)

In [45]:

# Identify lists of timestamps for different categories
bs_times = set(
    df_pf.query("variable == 'SF_avg_1m_ue'").query("value > 0").time
).union(
    set(df_pf.query("variable == 'SF_avg_2m_ue'").query("value > 0").time)
)
nobs_times = set(df_pf.time).difference(bs_times)

ri_stable_times = df_pf.query("variable == 'Ri_3m_c'").query("value > 0.25").time
ri_unstable_times = df_pf.query("variable == 'Ri_3m_c'").query("value < -0.01").time
ri_neutral_times = df_pf.query("variable == 'Ri_3m_c'").query("value >= -0.01").query("value <= 0.25").time

tgrad_stable_times = df_pf.query("variable == 'temp_gradient_3m_c'").query("value > 0.01").time
tgrad_unstable_times = df_pf.query("variable == 'temp_gradient_3m_c'").query("value < -0.01").time
tgrad_neutral_times = df_pf.query("variable == 'temp_gradient_3m_c'").query("value >= -0.01").query("value <= 0.01").time

In [46]:
def w_diurnal_chart(src, title):
    return alt.layer(
        line, plot.encode(alt.Y('value:Q').scale(domain=[-0.05,0.05])), data = src
    ).properties(width = 150, height = 150).properties(title=title)

def w_diurnal_df(filter_times):
    src_pfmp = df.query("type == 'pfmp'").set_index('time').query("tower == 'c'")
    src_pfmp = src_pfmp[src_pfmp.index.isin(filter_times)]
    src_pfmp = src_pfmp.groupby([src_pfmp.index.floor('30T').time, 'height', 'tower', 'type'])[['value']].median()
    src_pfmp.index = src_pfmp.index.set_names(['time','height','tower', 'type'])
    src_pfmp = src_pfmp.reset_index()
    src_pfmp['time'] = src_pfmp['time'].apply(lambda t: dt.datetime.combine(dt.date(2023,1,1),t))
    return src_pfmp

(
w_diurnal_chart(w_diurnal_df(df_pf.time), 'All times') |\
w_diurnal_chart(w_diurnal_df(ri_stable_times), 'Stable times') |\
w_diurnal_chart(w_diurnal_df(ri_unstable_times), 'Unstable times') |\
w_diurnal_chart(w_diurnal_df(ri_neutral_times), 'Neutral times')
).resolve_scale(y='shared', x='shared').configure_title(anchor='middle').interactive().display(renderer='svg')

/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_522/991269909.py:9: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  src_pfmp = src_pfmp.groupby([src_pfmp.index.floor('30T').time, 'height', 'tower', 'type'])[['value']].median()
/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_522/991269909.py:9: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  src_pfmp = src_pfmp.groupby([src_pfmp.index.floor('30T').time, 'height', 'tower', 'type'])[['value']].median()
/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_522/991269909.py:9: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  src_pfmp = src_pfmp.groupby([src_pfmp.index.floor('30T').time, 'height', 'tower', 'type'])[['value']].median()
/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_522/991269909.py:9: FutureWarning: 'T' is deprecated and will be 

alt.HConcatChart(...)

In [47]:
def w_diurnal_chart(src, title):
    return alt.layer(
        line, plot.encode(alt.Y('value:Q').scale(domain=[-0.05,0.05])), data = src
    ).properties(width = 150, height = 150).properties(title=title)

def w_diurnal_df(filter_times):
    src_pfmp = df.query("type == 'pfmp'").set_index('time').query("tower == 'c'")
    src_pfmp = src_pfmp[src_pfmp.index.isin(filter_times)]
    src_pfmp = src_pfmp.groupby([src_pfmp.index.floor('30T').time, 'height', 'tower', 'type'])[['value']].median()
    src_pfmp.index = src_pfmp.index.set_names(['time','height','tower', 'type'])
    src_pfmp = src_pfmp.reset_index()
    src_pfmp['time'] = src_pfmp['time'].apply(lambda t: dt.datetime.combine(dt.date(2023,1,1),t))
    return src_pfmp

(
w_diurnal_chart(w_diurnal_df(df_pf.time), 'All times') |\
w_diurnal_chart(w_diurnal_df(tgrad_stable_times), 'Stable times') |\
w_diurnal_chart(w_diurnal_df(tgrad_unstable_times), 'Unstable times') |\
w_diurnal_chart(w_diurnal_df(tgrad_neutral_times), 'Neutral times')
).resolve_scale(y='shared', x='shared').configure_title(anchor='middle').interactive().display(renderer='svg')

/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_522/111946928.py:9: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  src_pfmp = src_pfmp.groupby([src_pfmp.index.floor('30T').time, 'height', 'tower', 'type'])[['value']].median()
/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_522/111946928.py:9: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  src_pfmp = src_pfmp.groupby([src_pfmp.index.floor('30T').time, 'height', 'tower', 'type'])[['value']].median()
/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_522/111946928.py:9: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  src_pfmp = src_pfmp.groupby([src_pfmp.index.floor('30T').time, 'height', 'tower', 'type'])[['value']].median()
/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_522/111946928.py:9: FutureWarning: 'T' is deprecated and will be 

alt.HConcatChart(...)

In [48]:
upvalley_wind_times = df_pfmp[df_pfmp.variable == 'dir_3m_c'].query("value < 152").query("value > 112").time.values
downvalley_wind_times = df_pfmp[df_pfmp.variable == 'dir_3m_c'].query("value < 332").query("value > 292").time.values
len(upvalley_wind_times),len(downvalley_wind_times)
(
w_diurnal_chart(w_diurnal_df(upvalley_wind_times), 'Upvalley winds') |\
w_diurnal_chart(w_diurnal_df(downvalley_wind_times), 'Downvalley winds')
).display(renderer='svg')

/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_522/111946928.py:9: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  src_pfmp = src_pfmp.groupby([src_pfmp.index.floor('30T').time, 'height', 'tower', 'type'])[['value']].median()
/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_522/111946928.py:9: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  src_pfmp = src_pfmp.groupby([src_pfmp.index.floor('30T').time, 'height', 'tower', 'type'])[['value']].median()


alt.HConcatChart(...)

In [51]:
def w_diurnal_chart(src, title):
    return alt.layer(
        line, 
        alt.Chart().mark_line().encode(
            alt.X('hours(time):T'),
            alt.Y('mean(value):Q').title('Vertical velocity (m/s)'),
            alt.Color('height:O').scale(scheme='turbo'),
            strokeDash='tower'
        ), 
        data = src
    ).properties(width = 300, height = 300)

def w_diurnal_df(filter_times):
    src_pfmp = df.query("type == 'pfmp'").set_index('time')
    src_pfmp = src_pfmp[src_pfmp.index.isin(filter_times)]
    src_pfmp = src_pfmp.groupby([src_pfmp.index.floor('30T').time, 'height', 'tower', 'type'])[['value']].median()
    src_pfmp.index = src_pfmp.index.set_names(['time','height','tower', 'type'])
    src_pfmp = src_pfmp.reset_index()
    src_pfmp['time'] = src_pfmp['time'].apply(lambda t: dt.datetime.combine(dt.date(2023,1,1),t))
    return src_pfmp
(
w_diurnal_chart(w_diurnal_df(upvalley_wind_times).query("height > 1"), 'Upvalley winds').properties(width=200, height=200, title='Upvalley winds') |\
w_diurnal_chart(w_diurnal_df(downvalley_wind_times).query("height > 1"), 'Downvalley winds').properties(width=200, height=200, title='Downvalley winds')
).resolve_scale(color='shared', x='shared', y='shared').display(renderer='svg')

/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_522/306568817.py:16: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  src_pfmp = src_pfmp.groupby([src_pfmp.index.floor('30T').time, 'height', 'tower', 'type'])[['value']].median()
/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_522/306568817.py:16: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  src_pfmp = src_pfmp.groupby([src_pfmp.index.floor('30T').time, 'height', 'tower', 'type'])[['value']].median()


alt.HConcatChart(...)

In [15]:
start_date = '20221101'
end_date = '20230619'

def daily_w_cycle(src, title, ylim):
    return alt.Chart(
        src[src.variable.isin([
            'w_2m_c', 'w_3m_c', 'w_5m_c', 'w_10m_c', 'w_15m_c', 'w_20m_c',
            'w_3m_ue','w_10m_ue',
            'w_3m_uw','w_10m_uw',
            'w_3m_d','w_10m_d',
        ])]
    ).mark_line().encode(
        alt.X('hoursminutes(time):T'),
        alt.Y('median(value):Q').title('Wind speed (m/s)').scale(domain=ylim),
        alt.Color('height:O').scale(scheme='sinebow'),
        alt.StrokeDash('tower:N'),
        tooltip='variable',
    ).properties(width = 200, height = 200, title=title)

(

    daily_w_cycle(
        pd.read_parquet(f"tidy_df_{start_date}_{end_date}_planar_fit.parquet"),
        'SRLEC',
        ylim=[-0.10, 0.05]
    ) | daily_w_cycle(
        pd.read_parquet(f"tidy_df_{start_date}_{end_date}_noplanar_fit.parquet"),
        'Geographic',
        ylim=[-0.10, 0.05]
    ) | 
    daily_w_cycle(
        pd.read_parquet(f"tidy_df_{start_date}_{end_date}_planar_fit_multiplane.parquet"),
        'Streamwise',
        ylim=[-0.10, 0.05]
    )
).properties(title='Vertical wind speed, different coordinate systems').display(renderer='svg')

alt.HConcatChart(...)